In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
train_file = pd.ExcelFile('Final_Train.xlsx')
sample_submission = pd.ExcelFile('Sample_submission.xlsx')

In [3]:
train_data = train_file.parse('Sheet1')
smpl_data = sample_submission.parse('Sheet1')

In [4]:
train_data.isnull().sum()/len(train_data)*100

Qualification          0.000000
Experience             0.000000
Rating                55.393390
Place                  0.419393
Profile                0.000000
Miscellaneous_Info    43.952357
Fees                   0.000000
dtype: float64

## Case 1: We will remove columns with more than 40% missing values

In [5]:
rel_features = ['Qualification', 'Experience', 'Place', 'Profile', 'Fees']

In [6]:
rel_data = train_data[rel_features]

In [7]:
rel_data.isnull().sum()/len(rel_data)*100

Qualification    0.000000
Experience       0.000000
Place            0.419393
Profile          0.000000
Fees             0.000000
dtype: float64

## Removing rows with empty values

In [8]:
rel_data = rel_data.dropna()

## Extracting the no. of years from Experience variable

In [9]:
rel_data['Experience'] = rel_data['Experience'].loc[rel_data['Experience'].notnull()].apply(lambda x: re.search(r'\d+', x).group(0)).astype(int)

In [10]:
x = rel_data.Qualification.str.split(r',', expand=True).stack().reset_index(level=1, drop=True).to_frame('Qualification')

In [11]:
y = pd.get_dummies(x, columns=['Qualification']).groupby(level=0).sum()

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
le = LabelEncoder()

In [14]:
rel_data['Profile'] = le.fit_transform(rel_data['Profile'])

In [15]:
rel_data['Place'] = le.fit_transform(rel_data['Place'])

## Using One hot encoding

In [16]:
# import category_encoders as ce
# ohe = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)
# X_train_ohe = ohe.fit_transform(rel_data)
# X_train_ohe

## Transformed dataset

In [17]:
trans_data = pd.concat([y, rel_data.drop(["Qualification"], axis=1)], axis=1)

In [18]:
trans_data.shape

(5936, 907)

## Splitiing Train data to build model

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X = trans_data.drop(['Fees'], axis = 1)

In [21]:
y = trans_data[['Fees']]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

## Preparing Model using XGBoost

In [23]:
from xgboost import XGBRegressor
regressor  = XGBRegressor()

In [24]:
regressor.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [25]:
Y_pred_train = regressor.predict(X_train)

In [26]:
y_pred = regressor.predict(X_test)

In [27]:
def rmse(y_pred,y_test) :
    error = np.square(np.log10(y_pred +1) - np.log10(y_test +1)).mean() ** 0.5
    Acc = 1 - error
    return Acc

In [28]:
print("Accuracy attained on Training Set = ",rmse(Y_pred_train, np.array(y_train).reshape(-1)))
print("Accuracy attained on Test Set = ",rmse(y_pred,np.array(y_test).reshape(-1)))

Accuracy attained on Training Set =  0.719939626520241
Accuracy attained on Test Set =  0.7172571512401777


## Using decision Tree Classifier

In [29]:
from sklearn.tree import DecisionTreeRegressor

In [30]:
X_dtr = trans_data.drop(['Fees'], axis = 1)
y_dtr = trans_data[['Fees']]

In [31]:
dtr = DecisionTreeRegressor()

In [32]:
X_train_dtr, X_test_dtr, y_train_dtr, y_test_dtr = train_test_split(X_dtr, y_dtr, test_size = 0.2)

In [33]:
dtr.fit(X_train_dtr,y_train_dtr)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [34]:
Y_pred_dtr = dtr.predict(X_test_dtr)

In [35]:
Y_pred_dtr_train = dtr.predict(X_train_dtr)

In [36]:
print("Accuracy attained on Training Set = ",rmse(Y_pred_dtr_train, np.array(y_train).reshape(-1)))
print("Accuracy attained on Test Set = ",rmse(Y_pred_dtr,np.array(y_test).reshape(-1)))

Accuracy attained on Training Set =  0.5826895730679382
Accuracy attained on Test Set =  0.5741320260019956


In [37]:
feature_importances = pd.DataFrame(dtr.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance'])

In [38]:
impt_features = [ feature for feature in feature_importances.index if feature_importances.loc[feature].values > 0 ]

## XGBoost with important features

In [39]:
X_new = trans_data[impt_features]

In [40]:
y_new = trans_data[['Fees']]

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2)

In [42]:
regressor.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [43]:
Y_pred_train = regressor.predict(X_train)

In [44]:
y_pred = regressor.predict(X_test)

In [45]:
print("Accuracy attained on Training Set = ",rmse(Y_pred_train, np.array(y_train).reshape(-1)))
print("Accuracy attained on Test Set = ",rmse(y_pred,np.array(y_test).reshape(-1)))

Accuracy attained on Training Set =  0.7226920039320626
Accuracy attained on Test Set =  0.7061346501346604


## Predicting on actual test data

In [46]:
test_file = pd.ExcelFile('Final_Test.xlsx')
test_data = test_file.parse('Sheet1')

In [47]:
rel_test_features = ['Qualification', 'Experience', 'Place', 'Profile']

In [48]:
test_data = test_data[rel_test_features]

In [50]:
test_data.isnull().sum()/len(test_data)*100

Qualification    0.000000
Experience       0.000000
Place            0.301963
Profile          0.000000
dtype: float64

In [51]:
test_data['Experience'] = test_data['Experience'].loc[test_data['Experience'].notnull()].apply(lambda x: re.search(r'\d+', x).group(0)).astype(int)

In [52]:
le = LabelEncoder()

In [53]:
z = le.fit_transform(test_data['Place'].loc[test_data['Place'].notnull()])
z_median = np.median(y)

In [54]:
test_data['Place'].loc[test_data['Place'].notnull()] = z

C:\Users\arindmishra001\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [55]:
test_data['Place'].loc[test_data['Place'].isnull()] = z_median

In [56]:
test_data['Profile'] = le.fit_transform(test_data['Profile'])

In [57]:
x1 = test_data.Qualification.str.split(r',', expand=True).stack().reset_index(level=1, drop=True).to_frame('Qualification')

In [58]:
y1 = pd.get_dummies(x1, columns=['Qualification']).groupby(level=0).sum()

In [86]:
y1 = y1.astype(int)

In [92]:
trans_test_data = pd.concat([y1, test_data.drop(["Qualification"], axis=1)], axis=1)

In [93]:
trans_test_data['Place'] = trans_test_data['Place'].astype(int)

In [101]:
data_new = pd.DataFrame()
for column in X_train.columns:
    if column in trans_test_data:
        data_new[column] = trans_test_data[column]
    else:
        data_new[column]=0

In [105]:
data_new.fillna(0)

,Qualification_ Advance Diploma in Nutrition and Dietetics,Qualification_ Advance Fellowship in Laser & Dermatosurgery,Qualification_ Advanced Certification in Clinical Diabetology,Qualification_ Advanced Endotontic Course On Root Canal,Qualification_ Advanced Trauma Life Support (ATLS),Qualification_ BAMS,Qualification_ BDS,Qualification_ BEMS,Qualification_ BHMS,Qualification_ BSc - Zoology,...,Qualification_MS - General Surgery,Qualification_MS - Otorhinolaryngology,Qualification_MSc,Qualification_PG Diploma in Child Health,Qualification_PhD - Orthodontics & Dentofacial Orthopaedics,Qualification_UK),Qualification_VD & Leprosy,Experience,Place,Profile
0,0.0,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,35,142,4
1,0.0,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,31,580,3
2,0.0,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,40,212,2
3,0.0,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,374,0
4,0.0,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,16,231,1
5,0.0,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,14,554,1
6,0.0,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,23,134,3
7,0.0,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,9,31,1
8,0.0,0.0,0.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,11,51,0
9,0.0,0.0,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,44,323,5


In [103]:
len(X_train.columns)

386

In [106]:
y_pred_test = regressor.predict(data_new)

In [63]:
y_pred_test = np.round(y_pred_test).astype(int)

In [108]:
pd.DataFrame(y_pred_test, columns = ['Fees']).to_excel("submission3.xlsx", index = False)

In [107]:
y_pred_test

array([321.2835 , 325.93842, 327.67404, ..., 332.6712 , 255.50018,
       394.79004], dtype=float32)